# PROJECT 2: Investigating the history of number words

## Dependencies:
1. It is assumed that the user has anaconda installed on their system
2. Additional from anaconda, a python module named 'distance' is required to run this notebook. It can be installed using the following pip command: <br>
    pip install distance
   





In [1]:
#Loading Dependencies
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
import math
import random
from distance import levenshtein as edit_d, hamming, nlevenshtein as nedit_d, jaccard

random.seed(1234)

#Data_Paths:
data_path = 'csv_files/'
metadata = 'Languages_indo_euro.csv'

Let's store all the language datasets in a dictionary so that we can fetch them later on easily

In [2]:
#Store all languages datasets in a dictionary 
lang_data = {file[18:-4]: pd.read_csv(data_path + file, encoding='utf8') for file in listdir(data_path)}

#No. of languages:
print("Number of languages: %d" % len(lang_data))

#Indo_European metadata
indo_euro = pd.read_csv(metadata, encoding='utf8')
print("Number of indo european languages: %d" % len(indo_euro))

Number of languages: 7221
Number of indo european languages: 414


## Defining some functions to be used later for similarity calculation.<br>
Each function's description is described within the function

In [3]:
def ancient_langs(data):
    '''
    Input: asjp languages metadata 
    Output: a list of ancient language names
    '''
    #store names of languages that are extinct
    names = data[data.long_extinct == True].name.values
    
    #replace spaces by '_' and changes everything to uppercase
    names = [re.sub(r' ', r'_', name).upper() for name in names] 
    
    print("%d ancient languages stored" % len(names))
    print(names)
    print("\n")
    
    return [(x, 'ancient') for x in names]
    
def recently_extinct(data):
    '''
    Input: asjp language metadata 
    Output: a list of names of recently extinct languages
    '''
    #store names of languages that are extinct
    names = data[data.recently_extinct == True].name.values
    
    #replace spaces by '_' and changes everything to uppercase
    names = [re.sub(r' ', r'_', name).upper() for name in names] 
    
    print("%d recently extinct languages stored" % len(names))
    print(names)
    print("\n")
    
    return [(x, 'recently_extinct') for x in names]

def new_languages(data):
    '''
    Input: asjp language metadata 
    Output: a short subset of list of names of recent languages (non-extinct)
    '''
    random.seed(12)
    
    #store names of languages that are extinct
    names = data[(data.recently_extinct == False) & \
                (data.long_extinct == False)].name.values
    
    #Select random 10 language names (as the whole list is too long)
    random.shuffle(names)
    names = names[:13]
    
    #replace spaces by '_' and changes everything to uppercase
    names = [re.sub(r' ', r'_', name).upper() for name in names] 
    
    random.shuffle(names)
    
    print("%d new languages stored" % len(names))
    print(names)
    print("\n")
    
    return [(x, 'non_extinct') for x in names]

def vocab(language):
    '''
    Input: language name (a string) 
    Output: list of words in that language
    '''
    vocab = set(list(lang_data[language].Parameter_name.values))
    
    print("%d words of %s stored" % (len(vocab), language))
    
    return list(vocab)

def similarity_metric(metric, lang1, lang2, word):
    '''
    Input:  1. metric: Similarity function's name  : {'edit_dist', 'jaccard'}
            2. lang1: first language's name
            3. lang2: second language's name
            4. word: word to be compared
            
    Output: Value of similarity metric calculated on the word b/w lang1 and lang2
    '''
    values_lang1 = lang_data[lang1][lang_data[lang1].Parameter_name == word].Value.values
    values_lang2 = lang_data[lang2][lang_data[lang2].Parameter_name == word].Value.values
    #If multiple values, pick the one with smallest length
    index1 = list(values_lang1).index(min(values_lang1, key=len))
    index2 = list(values_lang2).index(min(values_lang2, key=len))
    
    if metric == 'edit_d':
        return edit_d(values_lang1[index1], values_lang2[index2])
    
    elif metric == 'nedit_d':  #Normalized edit distance
        return nedit_d(values_lang1[index1], values_lang2[index2], method=2) #longest alignment
        
    elif metric == 'norm_hamming':
        return hamming(values_lang1[index1], values_lang2[index2], normalized = True)
    
    elif metric == 'jaccard':
        return jaccard(values_lang1[index1], values_lang2[index2])
        
    else:
        return None

## Three clusters of language chosen: Ancient, Recently Extinct, Non-extinct

The names of these languages are stored in lists to be fetched later on <br>


In [4]:
#Store the names of languages in a list 
ancient_l = ancient_langs(indo_euro)
rec_ext_l = recently_extinct(indo_euro)
new_lang_l = new_languages(indo_euro)

#Common words
#words = vocab('ENGLISH')
words = ['one', 'two', 'I', 'you', 'we', 'person', 'fish', 'dog', \
         'louse', 'tree', 'leaf', 'skin', 'blood', 'bone', 'horn', \
        'ear', 'eye', 'nose', 'tooth', 'tongue', 'knee', 'hand', \
        'breast', 'liver', 'drink', 'see', 'hear', 'die', 'come',
        'sun', 'star', 'water', 'stone', 'fire', 'path',\
        'mountain', 'night', 'full', 'new', 'name']

21 ancient languages stored
['HITTITE', 'PALAIC', 'ARMENIAN_CLASSICAL', 'OLD_PRUSSIAN', 'CORNISH', 'GOTHIC', 'OLD_ENGLISH', 'OLD_FRISIAN', 'OLD_HIGH_GERMAN', 'OLD_LOW_FRANCONIAN', 'OLD_NORSE', 'OLD_SAXON', 'GREEK_ANCIENT', 'ILLYRIAN', 'PALI', 'SANSKRIT', 'AVESTAN', 'AVESTAN_2', 'OLD_PERSIAN', 'LATIN', 'OLD_CHURCH_SLAVONIC']


13 recently extinct languages stored
['MANX', 'OLD_IRISH', 'WELSH_ROMANI', 'DALMATIAN', 'EMILIANO_CARPIGIANO', 'EMILIANO_FERRARESE', 'EMILIANO_REGGIANO', 'ROMAGNOL_RAVENNATE', 'COMMON_TOCHARIAN', 'TOCHARIAN_A', 'TOCHARIAN_B', 'BERBICE_DUTCH_CREOLE', 'NEGERHOLLANDS']


13 new languages stored
['KASHMIRI', 'KOSOVO_ARLI_ROMANI', 'PARGAM_NISAR_KHOWAR', 'DOMAAKI', 'SANDNES_NORWEGIAN', 'KASHUBIAN', 'GULLAH', 'ASHRET_PHALURA', 'FRENCH', 'JUDEO_TAT_2', 'ENGLISH', 'PALAS_SHINA', 'CHAMAN_PASHTO']




## Metric 1: EDIT DISTANCE
### Dataframe to compare EDIT DISTANCES between phonemes of words in different languages

In [5]:
all_langs = new_lang_l + rec_ext_l + ancient_l

#Multi index for pandas dataframe
index_adv = pd.MultiIndex.from_tuples(all_langs, names=['language', 'cluster'])

Calculating edit_distance between each pair of words between different languages

In [6]:
#Create an empty dataframe with columns as words and multi index (language, cluster)
df = pd.DataFrame(columns = words, index = index_adv, dtype=float)

for (lang, cluster) in all_langs:
    for word in words:
        try:
            df.loc[(lang, cluster),word] = similarity_metric('edit_d', 'ENGLISH', lang, word)
        except:
            pass   #for words not found in a particular language               

#### A look at EDIT_DISTANCE data frame

The NaN values mean that the particular word was not found in that language

In [7]:
df

,,one,two,I,you,we,person,fish,dog,louse,tree,...,star,water,stone,fire,path,mountain,night,full,new,name
language,cluster,,,,,,,,,,,,,,,,,,,,,
KASHMIRI,non_extinct,3.0,3.0,3.0,2.0,3.0,5.0,3.0,3.0,2.0,3.0,...,3.0,4.0,4.0,3.0,3.0,7.0,3.0,5.0,1.0,2.0
KOSOVO_ARLI_ROMANI,non_extinct,3.0,2.0,2.0,1.0,4.0,6.0,6.0,5.0,3.0,5.0,...,6.0,4.0,4.0,4.0,5.0,6.0,3.0,7.0,3.0,3.0
PARGAM_NISAR_KHOWAR,non_extinct,3.0,1.0,3.0,1.0,3.0,6.0,4.0,4.0,NaN,3.0,...,2.0,5.0,4.0,4.0,2.0,NaN,3.0,NaN,2.0,1.0
DOMAAKI,non_extinct,3.0,2.0,NaN,1.0,3.0,NaN,6.0,6.0,NaN,2.0,...,3.0,5.0,3.0,4.0,NaN,NaN,5.0,NaN,3.0,1.0
SANDNES_NORWEGIAN,non_extinct,2.0,1.0,1.0,NaN,2.0,5.0,2.0,3.0,1.0,1.0,...,4.0,3.0,4.0,4.0,3.0,7.0,2.0,NaN,2.0,3.0
KASHUBIAN,non_extinct,4.0,3.0,2.0,2.0,2.0,NaN,4.0,5.0,NaN,7.0,...,NaN,4.0,4.0,4.0,5.0,6.0,3.0,5.0,3.0,6.0
GULLAH,non_extinct,1.0,0.0,2.0,0.0,0.0,3.0,NaN,1.0,NaN,0.0,...,NaN,3.0,NaN,2.0,2.0,7.0,1.0,0.0,NaN,0.0
ASHRET_PHALURA,non_extinct,3.0,1.0,2.0,1.0,2.0,6.0,5.0,6.0,NaN,3.0,...,3.0,5.0,4.0,4.0,3.0,NaN,3.0,NaN,2.0,2.0
FRENCH,non_extinct,3.0,2.0,2.0,2.0,2.0,6.0,7.0,3.0,3.0,4.0,...,5.0,5.0,5.0,3.0,3.0,5.0,2.0,4.0,2.0,2.0


### Mean of Edit Distances for three clusters

In [8]:
#Missing Value imputations
df.fillna(df.median(), inplace=True)
df.groupby(level=['cluster']).mean()

,one,two,I,you,we,person,fish,dog,louse,tree,...,star,water,stone,fire,path,mountain,night,full,new,name
cluster,,,,,,,,,,,,,,,,,,,,,
ancient,3.047619,2.857143,2.333333,1.714286,2.333333,5.904762,3.904762,3.619048,3.952381,3.523810,...,2.904762,3.047619,3.285714,2.952381,3.428571,6.380952,3.190476,3.047619,2.904762,2.619048
non_extinct,2.615385,1.692308,1.923077,1.230769,2.307692,5.000000,3.846154,3.461538,3.153846,2.846154,...,3.230769,3.923077,3.846154,3.307692,2.769231,6.153846,2.769231,3.230769,2.230769,1.769231
recently_extinct,2.461538,1.923077,2.076923,1.384615,2.846154,5.692308,3.076923,3.076923,4.230769,4.000000,...,3.846154,3.538462,4.076923,3.153846,5.076923,5.615385,3.307692,3.230769,1.846154,1.461538


### Number words Vs Other words

In [9]:
number_words = list(df.columns[:2])
other_words = list(df.columns[2:])

df['number_words'] = df[number_words].mean(axis=1)
df['other_words'] = df[other_words].mean(axis=1)

df[['number_words', 'other_words']].groupby(level=['cluster']).mean()

,number_words,other_words
cluster,,
ancient,2.952381,3.635338
non_extinct,2.153846,3.313765
recently_extinct,2.192308,3.682186


## METRIC 2: Normalized Edit DISTANCE

#### Dataframe based on normalized Edit Distance:
We can analyze all the results with respect to normalized edit distance as well

In [10]:
#Create an empty dataframe with columns as words and multi index (language, cluster)
df = pd.DataFrame(columns = words, index = index_adv, dtype=float)

for (lang, cluster) in all_langs:
    for word in words:
        try:
            df.loc[(lang, cluster),word] = similarity_metric('nedit_d', 'ENGLISH', lang, word)
        except:
          pass   #for words not found in a particular language     
    
#Missing Value imputations
df.fillna(df.median(), inplace=True)
df.groupby(level=['cluster']).mean()

,one,two,I,you,we,person,fish,dog,louse,tree,...,star,water,stone,fire,path,mountain,night,full,new,name
cluster,,,,,,,,,,,,,,,,,,,,,
ancient,0.823413,0.801134,0.857143,0.663492,0.768254,0.884921,0.775661,0.897619,0.726190,0.804195,...,0.473621,0.601587,0.719841,0.658730,0.850000,0.894558,0.701587,0.614626,0.693651,0.606349
non_extinct,0.830769,0.641026,0.858974,0.615385,0.826923,0.791209,0.837179,0.826923,0.665385,0.699359,...,0.561172,0.784615,0.793956,0.765385,0.788462,0.798535,0.657692,0.753846,0.628205,0.463370
recently_extinct,0.737179,0.750000,0.897436,0.608974,0.801282,0.876374,0.862821,0.865385,0.876923,0.826190,...,0.689744,0.697436,0.828205,0.761538,0.929487,0.751374,0.726923,0.858974,0.583333,0.443590


In [11]:
number_words = list(df.columns[:2])
other_words = list(df.columns[2:])

df['number_words'] = df[number_words].mean(axis=1)
df['other_words'] = df[other_words].mean(axis=1)

df[['number_words', 'other_words']].groupby(level=['cluster']).mean()

,number_words,other_words
cluster,,
ancient,0.812273,0.749742
non_extinct,0.735897,0.747043
recently_extinct,0.743590,0.808606


## METRIC 3: Normalized Hamming

#### Dataframe based on normalized Hamming:
We can analyze all the results with respect to normalized hamming

In [12]:
#Create an empty dataframe with columns as words and multi index (language, cluster)
df = pd.DataFrame(columns = words, index = index_adv, dtype=float)

for (lang, cluster) in all_langs:
    for word in words:
        try:
            df.loc[(lang, cluster),word] = similarity_metric('norm_hamming', 'ENGLISH', lang, word)
        except:
          pass   #for words not found in a particular language     
    
#Missing Value imputations
df.fillna(df.median(), inplace=True)
df.groupby(level=['cluster']).mean()

,one,two,I,you,we,person,fish,dog,louse,tree,...,star,water,stone,fire,path,mountain,night,full,new,name
cluster,,,,,,,,,,,,,,,,,,,,,
ancient,0.968254,0.523810,0.952381,0.595238,0.500000,0.920635,1.000000,0.698413,1.000000,0.968254,...,0.952381,0.409524,0.726190,0.714286,0.666667,0.857143,0.797619,0.539683,0.500000,0.365079
non_extinct,0.871795,0.500000,0.884615,0.615385,0.576923,0.839744,0.923077,0.666667,0.884615,0.743590,...,0.923077,0.384615,0.711538,0.711538,0.717949,0.791209,0.673077,0.589744,0.461538,0.358974
recently_extinct,0.923077,0.615385,0.923077,0.576923,0.538462,0.923077,0.948718,0.692308,1.000000,1.000000,...,1.000000,0.553846,0.730769,0.769231,0.717949,0.857143,0.730769,0.794872,0.500000,0.410256


In [13]:
number_words = list(df.columns[:2])
other_words = list(df.columns[2:])

df['number_words'] = df[number_words].mean(axis=1)
df['other_words'] = df[other_words].mean(axis=1)

df[['number_words', 'other_words']].groupby(level=['cluster']).mean()

,number_words,other_words
cluster,,
ancient,0.746032,0.713429
non_extinct,0.685897,0.667852
recently_extinct,0.769231,0.738245


## METRIC 4: Jaccard

#### Dataframe based on Jaccard


In [14]:
#Create an empty dataframe with columns as words and multi index (language, cluster)
df = pd.DataFrame(columns = words, index = index_adv, dtype=float)

for (lang, cluster) in all_langs:
    for word in words:
        try:
            df.loc[(lang, cluster),word] = similarity_metric('jaccard', 'ENGLISH', lang, word)
        except:
          pass   #for words not found in a particular language     
    
#Missing Value imputations
df.fillna(df.median(), inplace=True)
df.groupby(level=['cluster']).mean()

,one,two,I,you,we,person,fish,dog,louse,tree,...,star,water,stone,fire,path,mountain,night,full,new,name
cluster,,,,,,,,,,,,,,,,,,,,,
ancient,0.871032,0.845918,0.873016,0.768254,0.821429,0.886144,0.814683,0.880499,0.732937,0.835809,...,0.432540,0.656047,0.762528,0.719274,0.892744,0.901623,0.716837,0.620975,0.703175,0.586054
non_extinct,0.852564,0.705128,0.871795,0.692308,0.826923,0.782143,0.853114,0.856410,0.696154,0.725641,...,0.553266,0.815476,0.779670,0.783150,0.839103,0.807082,0.700641,0.756410,0.675641,0.567308
recently_extinct,0.798718,0.824359,0.923077,0.683333,0.839744,0.859463,0.874908,0.914652,0.856838,0.864927,...,0.647619,0.711355,0.869231,0.818590,0.950000,0.666300,0.779945,0.830037,0.652564,0.557692


In [15]:
number_words = list(df.columns[:2])
other_words = list(df.columns[2:])

df['number_words'] = df[number_words].mean(axis=1)
df['other_words'] = df[other_words].mean(axis=1)

df[['number_words', 'other_words']].groupby(level=['cluster']).mean()

,number_words,other_words
cluster,,
ancient,0.858475,0.769448
non_extinct,0.778846,0.770276
recently_extinct,0.811538,0.822428
